In [1]:
import sys
import os
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

C:\Users\Ananya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ananya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Ananya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

keras = tf.keras

In [3]:
def sequential_window_dataset(series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1))
    ds = ds.map(lambda window: (window[:-1], window[1:]))
    return ds.batch(1).prefetch(1)

In [4]:
# Read in data
forex = pd.read_csv('./GBPUSD.csv')

# Convert series into datetime type
forex.index = pd.to_datetime(forex['Date'])

# Save target series
series = forex['Close']

#Size of exchange rates
total_size = forex.shape[0]
print(total_size)

# getting 1 week of future predictions
n_days = 5

#Use 80% of data as training, rest 20% to Validate 
valid_split_index = int(total_size * 0.8)
x_train = forex.iloc[0:valid_split_index].Close
x_valid = forex.iloc[valid_split_index:-n_days].Close

# test set is last 5 dates from the data collected
x_test = forex.iloc[-n_days:total_size].Close

4852


In [5]:
# Reshape the datsets
x_train_values = x_train.values.reshape(-1, 1)
x_valid_values = x_valid.values.reshape(-1, 1)
x_test_values = x_test.values.reshape(-1, 1)

# Create a Scaler Object
x_train_scaler = MinMaxScaler(feature_range=(0, 1))

# normalize training data
normalized_x_train = x_train_scaler.fit_transform(x_train_values)

# normalize validation data
normalized_x_valid = x_train_scaler.transform(x_valid_values)

# normalize test data
normalized_x_test = x_train_scaler.transform(x_test_values)

# Normalized to training data
forex_normalized_to_traindata = x_train_scaler.transform(series.values.reshape(-1, 1))

In [6]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

## Build and Train LSTM Model

In [ ]:
# reset any stored data
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# set window size and create input batch sequence
window_size = 20
train_set = sequential_window_dataset(normalized_x_train, window_size)
valid_set = sequential_window_dataset(normalized_x_valid, window_size)

# create model
model = keras.models.Sequential([
  keras.layers.LSTM(100, return_sequences=True, stateful=True,
                         batch_input_shape=[1, None, 1]),
  keras.layers.LSTM(100, return_sequences=True, stateful=True),
  keras.layers.Dense(1),
])

# set optimizer
optimizer = keras.optimizers.Nadam(lr=1e-4)

# compile model
model.compile(loss="mean_squared_error",
              optimizer=optimizer,
              metrics=["mae"])

# reset states
reset_states = ResetStatesCallback()

#set up save best only checkpoint
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "lstm_model", save_best_only=True)

early_stopping = keras.callbacks.EarlyStopping(patience=50)

# fit model
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint, reset_states], verbose=0)

## Make Predictions

In [8]:
# recall best model
model = keras.models.load_model("lstm_model")

In [27]:
# make predictions
rnn_forecast = model.predict(normalized_x_test[np.newaxis,:])
rnn_forecast = rnn_forecast.flatten()

rnn_unscaled_forecast = x_train_scaler.inverse_transform(rnn_forecast.reshape(-1,1)).flatten()
rnn_unscaled_forecast

array([1.1860942, 1.1837622, 1.1865613, 1.1759346, 1.1772939],
      dtype=float32)

# Trading Algorithm

In [69]:
# let us start with 100 GBP already being invested and 100 USD at our disposal for more investment
last_week_closing_price = x_valid[-1]

# all prices are in USD
invested = 100 * last_week_closing_price
capital = 100
original_capital = capital
original_invested = invested


risk_margin = 0.001

print("Starting the trading for this week using predicted values for 5 days.... \n")
# using prediicted prices
for price_number in range(0, len(rnn_unscaled_forecast)):
    if price_number == 0:
        last_closing_price = last_week_closing_price
    else: 
        last_closing_price = rnn_unscaled_forecast[price_number-1]
        
    rate = rnn_unscaled_forecast[price_number]
    
    price_change_rate = (rate - last_closing_price)/last_closing_price
    
    if price_change_rate > risk_margin:
        # consider SELL
        print("For ", x_test.index[price_number], " : Selling")
        current_units = invested / last_closing_price
        units_sell = 0.1 * current_units
        invested = invested - (units_sell * rate)
        capital = capital + (units_sell * rate)
    
    elif price_change_rate < -risk_margin:
        # consider BUY
        print("For ", x_test.index[price_number], " : Buying")
        current_units = invested / last_closing_price
        units_sell = 0.1 * current_units
        invested = invested + (units_sell * rate)
        capital = capital - (units_sell * rate)
        
    else:
        # HOLD
        print("For ", x_test.index[price_number], " : Holding")
        
print("\n###################################################\n")
        
new_capital = (capital - original_capital)
percent_new_capital = (new_capital/original_capital) * 100
print("Original capital = ", original_capital, ", and New Capital = ", capital)
print("Total profit/loss generated = ", new_capital)
print("% change in current capital = ", percent_new_capital)
new_invested = (invested - original_invested)
percent_new_invested = (new_invested/original_invested) * 100
print("\nOriginal investment = ", original_invested, ", and New Investment = ", invested)
print("Total increase/decrease in potfolio generated = ", new_invested)
print("% change in current investment = ", percent_new_invested)
print("\nOriginal units held = ", original_invested/last_week_closing_price, ", and New units held = ", invested_actual/rnn_unscaled_forecast[-1])

Starting the trading for this week using predicted values for 5 days.... 

For  2022-08-24 00:00:00  : Selling
For  2022-08-25 00:00:00  : Buying
For  2022-08-26 00:00:00  : Selling
For  2022-08-29 00:00:00  : Buying
For  2022-08-30 00:00:00  : Selling

###################################################

Original capital =  100 , and New Capital =  114.10991288293145
Total profit/loss generated =  14.109912882931454
% change in current capital =  14.109912882931452

Original investment =  117.66505241394043 , and New Investment =  103.55513953100898
Total increase/decrease in potfolio generated =  -14.109912882931454
% change in current investment =  -11.991591890252517

Original units held =  100.0 , and New units held =  87.98040316040468


In [70]:
# let us start with 100 GBP already being invested and 100 USD at our disposal for more investment
last_week_closing_price = x_valid[-1]

# all prices are in USD
invested_actual = 100 * last_week_closing_price
capital_actual = 100
original_capital_actual = capital_actual
original_invested_actual = invested_actual


risk_margin = 0.001

print("Starting the trading for this week using actual values for 5 days.... \n")
# using prediicted prices
for price_number in range(0, len(x_test)):
    if price_number == 0:
        last_closing_price_actual = last_week_closing_price
    else: 
        last_closing_price_actual = x_test[price_number-1]
        
    rate_actual = x_test[price_number]
    
    price_change_rate_actual = (rate_actual - last_closing_price_actual)/last_closing_price_actual
    
    if price_change_rate_actual > risk_margin:
        # consider SELL
        print("For ", x_test.index[price_number], " : Selling")
        current_units = invested_actual / last_closing_price_actual
        units_sell = 0.1 * current_units
        invested_actual = invested_actual - (units_sell * rate_actual)
        capital_actual = capital_actual + (units_sell * rate_actual)
    
    elif price_change_rate_actual < -risk_margin:
        # consider BUY
        print("For ", x_test.index[price_number], " : Buying")
        current_units = invested_actual / last_closing_price_actual
        units_sell = 0.1 * current_units
        invested_actual = invested_actual + (units_sell * rate)
        capital_actual = capital_actual - (units_sell * rate)
        
    else:
        # HOLD
        print("For ", x_test.index[price_number], " : Holding")
        
print("\n###################################################\n")
        
new_capital_actual = (capital_actual - original_capital_actual)
percent_new_capital_actual = (new_capital_actual/original_capital_actual) * 100
print("Original capital = ", original_capital_actual, ", and New Capital = ", capital_actual)
print("Total profit/loss generated = ", new_capital_actual)
print("% change in current capital = ", percent_new_capital_actual)
new_invested_actual = (invested_actual - original_invested_actual)
percent_new_invested_actual = (new_invested_actual/original_invested_actual) * 100
print("\nOriginal investment = ", original_invested_actual, ", and New Investment = ", invested_actual)
print("Total increase/decrease in potfolio generated = ", new_invested_actual)
print("% change in current investment = ", percent_new_invested_actual)
print("\nOriginal units held = ", original_invested_actual/last_week_closing_price, ", and New units held = ", invested_actual/x_test[-1])


Starting the trading for this week using actual values for 5 days.... 

For  2022-08-24 00:00:00  : Selling
For  2022-08-25 00:00:00  : Buying
For  2022-08-26 00:00:00  : Selling
For  2022-08-29 00:00:00  : Buying
For  2022-08-30 00:00:00  : Selling

###################################################

Original capital =  100 , and New Capital =  114.08626074618046
Total profit/loss generated =  14.086260746180457
% change in current capital =  14.086260746180457

Original investment =  117.66505241394043 , and New Investment =  103.57879166775997
Total increase/decrease in potfolio generated =  -14.086260746180457
% change in current investment =  -11.971490648409027

Original units held =  100.0 , and New units held =  88.35892230318701
